In [154]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

pd.set_option('display.max_columns', None)

In [2]:
mintic = glob.glob('../data/*clean.csv')
len(mintic)

3

In [3]:
for file in mintic:
    print(file)

../data\comp_dim_clean.csv
../data\mdm_clean.csv
../data\mintic_investments_clean.csv


In [4]:
terridata = glob.glob('../data/TerriData*csv')
len(terridata)

18

In [5]:
for file in terridata:
    print(file)

../data\TerriData_ambiente.csv
../data\TerriData_censo_2005_y_proyecciones_dane.csv
../data\TerriData_conflicto_armado_y_seguridad_ciudadana.csv
../data\TerriData_demografia_y_poblacion.csv
../data\TerriData_descripcion_general.csv
../data\TerriData_economia.csv
../data\TerriData_educacion.csv
../data\TerriData_finanzas_publicas.csv
../data\TerriData_justicia_y_derecho.csv
../data\TerriData_medicion_de_desempenio_municipal.csv
../data\TerriData_mercado_laboral.csv
../data\TerriData_ordenamiento_territorial.csv
../data\TerriData_percepcion_ciudadana.csv
../data\TerriData_pobreza.csv
../data\TerriData_presupuesto_general_de_la_nacion.csv
../data\TerriData_salud.csv
../data\TerriData_seguridad_integral_maritima_y_fluvial.csv
../data\TerriData_vivienda_y_acceso_a_servicios_publicos.csv


In [132]:
other = set(glob.glob('../data/*csv')) - set(terridata) - set(mintic)
other = list(other)
len(other)

2

In [133]:
for file in other:
    print(file)

../data\df_vars_modelamiento.csv
../data\df_merged_final_vars.csv


In [107]:
comp_dim = pd.read_csv(mintic[0])
mdm = pd.read_csv(mintic[1])
investment = pd.read_csv(mintic[2])

In [7]:
municipios = mdm.codigo.unique()
len(municipios)

1101

In [9]:
departamentos = investment.departame_cod.unique()
departamentos = list(departamentos)
departamentos.pop(departamentos.index('0'))
departamentos.pop(departamentos.index('in'))
departamentos = [int(i) for i in departamentos]
len(departamentos)

33

In [37]:
to_keep = ['departame_cod', 'municipio_cod', 'departame_nombre']
codes = investment.loc[investment.municipio_cod.isin(municipios)][to_keep].drop_duplicates()
len(codes)

1101

In [49]:
investment.head(1)

,anio_corte,fecha_corte,departame_nombre,departame_cod,municipio_nombre,municipio_cod,iniciativa,viceministerio,indicador,descripcion_indicador,inversion,beneficiarios,id
0,2020,2020-12-31,bogota,11,bogota_d.c.,11001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,773577049710,606,202011001


In [121]:
to_group = ['anio_corte', 'departame_cod', 'departame_nombre']
inv_dep = investment.loc[investment.anio_corte.gt(2014)
                         & investment.municipio_cod.isin(municipios)].groupby(to_group + ['viceministerio']).sum().reset_index()
inv_dep.head(2)

,anio_corte,departame_cod,departame_nombre,viceministerio,municipio_cod,inversion,beneficiarios,id
0,2015,11,bogota,conectividad,22002,3916083754,8,403022002
1,2015,11,bogota,transformacion,66006,20946777748,144013,1209066006


In [122]:
to_pivot = ['anio_corte', 'departame_cod', 'departame_nombre']
inv_dep = inv_dep.pivot(index = to_pivot, columns = 'viceministerio', values = 'inversion').reset_index()
inv_dep['inversion_total'] = inv_dep.conectividad + inv_dep.transformacion
inv_dep['anio_inversion'] = inv_dep.anio_corte
inv_dep.anio_corte = inv_dep.anio_corte + 1
inv_dep.head()

viceministerio,anio_corte,departame_cod,departame_nombre,conectividad,transformacion,inversion_total,anio_inversion
0,2016,11,bogota,3.916084e+09,2.094678e+10,2.486286e+10,2015
1,2016,13,bolivar,3.540095e+10,1.586348e+10,5.126443e+10,2015
2,2016,15,boyaca,3.204076e+10,1.176500e+10,4.380576e+10,2015
3,2016,17,caldas,1.629884e+10,1.082415e+10,2.712298e+10,2015
4,2016,18,caqueta,1.883932e+10,4.330740e+09,2.317006e+10,2015


In [108]:
mdm.head(1)

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,mdm,puesto_mdm_grupo_dotaciones,puesto_mdm_nacional,id
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,80.78,1,3,20165001


In [109]:
mdm = mdm.merge(codes, left_on = 'codigo', right_on = 'municipio_cod')
mdm.head(1)

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,mdm,puesto_mdm_grupo_dotaciones,puesto_mdm_nacional,id,departame_cod,municipio_cod,departame_nombre
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,80.78,1,3,20165001,5,5001,antioquia


In [110]:
mdm_dep = mdm.groupby(to_group).mean()['mdm'].reset_index()
mdm_dep.head()

,anio_corte,departame_cod,departame_nombre,mdm
0,2016,11,bogota,77.560000
1,2016,13,bolivar,38.773043
2,2016,15,boyaca,49.328537
3,2016,17,caldas,50.502963
4,2016,18,caqueta,44.633750


In [111]:
comp_dim.head(1)

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,...,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,id
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,0.559,49.26,51.7,0.737,...,0.214,0.97,0.92,13871,55.78,536,2.16,2374,9.55,20165001


In [112]:
comp_dim = comp_dim.merge(codes, left_on = 'codigo', right_on = 'municipio_cod')
comp_dim.head(1)

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,...,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,id,departame_cod,municipio_cod,departame_nombre
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,0.559,49.26,51.7,0.737,...,13871,55.78,536,2.16,2374,9.55,20165001,5,5001,antioquia


In [113]:
dim_dep = comp_dim.groupby(to_group).mean().reset_index()
dim_dep.head()

,anio_corte,departame_cod,departame_nombre,codigo,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,...,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,id,municipio_cod
0,2016,11,bogota,11001.000000,0.497000,54.090000,54.350000,0.564000,0.924000,0.956000,...,0.960000,0.960000,34364.000000,43.060000,1278.000000,1.600000,4738.000000,5.940000,2.016110e+08,11001.000000
1,2016,13,bolivar,13465.456522,0.334391,42.556739,45.795652,0.510957,0.822130,0.790543,...,0.568043,0.217609,44.000000,2.797609,8.934783,1.270217,18.239130,2.694348,2.016135e+08,13465.456522
2,2016,15,boyaca,15481.699187,0.490642,52.212520,51.290244,0.569244,0.773033,0.886130,...,0.533415,0.265203,26.821138,11.426911,0.869919,0.935528,25.642276,13.514228,2.016155e+08,15481.699187
3,2016,17,caldas,17456.888889,0.412630,47.271852,49.739259,0.472778,0.827259,0.923815,...,0.694444,0.454074,93.962963,9.064444,7.962963,2.106296,27.777778,5.685926,2.016175e+08,17456.888889
4,2016,18,caqueta,18417.937500,0.214812,43.844375,45.901250,0.387750,0.720562,0.727563,...,0.556875,0.531250,97.187500,13.487500,8.875000,2.468125,31.625000,5.510625,2.016184e+08,18417.937500


In [114]:
to_drop = ['codigo', 'id', 'municipio_cod']
dim_dep.drop(to_drop, axis = 'columns', inplace = True)
dim_dep.head(1)

,anio_corte,departame_cod,departame_nombre,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab
0,2016,11,bogota,0.497,54.09,54.35,0.564,0.924,0.956,11.73,1.0,0.211,0.96,0.96,34364.0,43.06,1278.0,1.6,4738.0,5.94


In [123]:
merge_dep = dim_dep.merge(mdm_dep, on = to_pivot).merge(inv_dep, on = to_pivot)
merge_dep.head()

,anio_corte,departame_cod,departame_nombre,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,...,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,mdm,conectividad,transformacion,inversion_total,anio_inversion
0,2016,11,bogota,0.497000,54.090000,54.350000,0.564000,0.924000,0.956000,11.730000,...,43.060000,1278.000000,1.600000,4738.000000,5.940000,77.560000,3.916084e+09,2.094678e+10,2.486286e+10,2015
1,2016,13,bolivar,0.334391,42.556739,45.795652,0.510957,0.822130,0.790543,22.100652,...,2.797609,8.934783,1.270217,18.239130,2.694348,38.773043,3.540095e+10,1.586348e+10,5.126443e+10,2015
2,2016,15,boyaca,0.490642,52.212520,51.290244,0.569244,0.773033,0.886130,17.035691,...,11.426911,0.869919,0.935528,25.642276,13.514228,49.328537,3.204076e+10,1.176500e+10,4.380576e+10,2015
3,2016,17,caldas,0.412630,47.271852,49.739259,0.472778,0.827259,0.923815,14.015926,...,9.064444,7.962963,2.106296,27.777778,5.685926,50.502963,1.629884e+10,1.082415e+10,2.712298e+10,2015
4,2016,18,caqueta,0.214812,43.844375,45.901250,0.387750,0.720562,0.727563,21.023125,...,13.487500,8.875000,2.468125,31.625000,5.510625,44.633750,1.883932e+10,4.330740e+09,2.317006e+10,2015


In [136]:
merged_mun = pd.read_csv(other[0])
merged_mun.head()

,anio_corte,municipio,codigo_municipio,departamento,grupo_dotaciones,categoria_de_ruralidad,mdm_x,codigo_departamento,inversion,inversion_conectividad,...,proyeccion_de_poblacion_entre_los_45_y_49_anios,proyeccion_de_poblacion_entre_los_50_y_54_anios,proyeccion_de_poblacion_entre_los_55_y_59_anios,proyeccion_de_poblacion_entre_los_5_y_9_anios,proyeccion_de_poblacion_entre_los_60_y_64_anios,proyeccion_de_poblacion_entre_los_65_y_69_anios,proyeccion_de_poblacion_entre_los_70_y_74_anios,proyeccion_de_poblacion_entre_los_75_y_79_anios,departamento_fixed_name,municipio_fixed_name
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,0.8078,5,24698428098,223686523,...,181158.0,195486.0,172662.0,149779.0,133648.0,101309.0,65029.0,46837.0,Antioquia,Medellín
1,2016,abejorral,5002,antioquia,g3,rural,0.4948,5,86075327,24371542,...,1064.0,1163.0,1238.0,1620.0,1037.0,726.0,506.0,396.0,Antioquia,Abejorral
2,2016,abriaqui,5004,antioquia,g3,rural_disperso,0.4494,5,254082597,163619212,...,137.0,140.0,126.0,165.0,99.0,77.0,46.0,35.0,Antioquia,Abriaquí
3,2016,alejandria,5021,antioquia,g2,rural,0.5494,5,116590322,24371542,...,242.0,241.0,208.0,236.0,155.0,121.0,85.0,61.0,Antioquia,Alejandría
4,2016,amaga,5030,antioquia,g2,intermedios,0.5188,5,186016729,24371542,...,1579.0,1521.0,1124.0,2756.0,950.0,685.0,570.0,418.0,Antioquia,Amagá


In [138]:
merged_mun.columns[0:25]

Index(['anio_corte', 'municipio', 'codigo_municipio', 'departamento',
       'grupo_dotaciones', 'categoria_de_ruralidad', 'mdm_x',
       'codigo_departamento', 'inversion', 'inversion_conectividad',
       'inversion_transformacion', 'vacunacion_pentavalente_0_1',
       'componente_de_resultados', 'componente_de_gestion',
       'gobierno_abierto_y_transparencia', 'ejecucion_de_recursos',
       'valor_agregado', 'valor_agregado_per_capita',
       'valor_agregado_per_capita_como_porcentaje_del_promedio_nacional',
       'tasa_de_violencias_basadas_en_genero',
       '%_de_ingresos_corrientes_destinados_a_funcionamiento',
       '%_del_gasto_total_destinado_a_inversion', 'capacidad_de_ahorro',
       'deficit_o_superavit_total', 'financiamiento'],
      dtype='object')

In [139]:
to_drop = ['municipio', 'codigo_municipio', 'departamento',
           'grupo_dotaciones', 'categoria_de_ruralidad', 'mdm_x',
           'inversion', 'inversion_conectividad',
           'inversion_transformacion']
merged_mun.drop(to_drop, axis = 'columns', inplace = True)
merged_mun.columns[0:25]

Index(['anio_corte', 'codigo_departamento', 'vacunacion_pentavalente_0_1',
       'componente_de_resultados', 'componente_de_gestion',
       'gobierno_abierto_y_transparencia', 'ejecucion_de_recursos',
       'valor_agregado', 'valor_agregado_per_capita',
       'valor_agregado_per_capita_como_porcentaje_del_promedio_nacional',
       'tasa_de_violencias_basadas_en_genero',
       '%_de_ingresos_corrientes_destinados_a_funcionamiento',
       '%_del_gasto_total_destinado_a_inversion', 'capacidad_de_ahorro',
       'deficit_o_superavit_total', 'financiamiento', 'funcionamiento',
       'gastos_corrientes', 'gastos_corrientes_per_capita',
       'gastos_de_capital_(inversion)', 'gastos_de_funcionamiento_per_capita',
       'gastos_totales', 'gastos_totales_per_capita',
       'indicador_de_desempenio_fiscal', 'ingresos_corrientes'],
      dtype='object')

In [144]:
to_group = ['anio_corte', 'codigo_departamento']
mean_var_mun = merged_mun.groupby(to_group).mean().reset_index()
mean_var_mun.head()

,anio_corte,codigo_departamento,vacunacion_pentavalente_0_1,componente_de_resultados,componente_de_gestion,gobierno_abierto_y_transparencia,ejecucion_de_recursos,valor_agregado,valor_agregado_per_capita,valor_agregado_per_capita_como_porcentaje_del_promedio_nacional,...,proyeccion_de_poblacion_entre_los_35_y_39_anios,proyeccion_de_poblacion_entre_los_40_y_44_anios,proyeccion_de_poblacion_entre_los_45_y_49_anios,proyeccion_de_poblacion_entre_los_50_y_54_anios,proyeccion_de_poblacion_entre_los_55_y_59_anios,proyeccion_de_poblacion_entre_los_5_y_9_anios,proyeccion_de_poblacion_entre_los_60_y_64_anios,proyeccion_de_poblacion_entre_los_65_y_69_anios,proyeccion_de_poblacion_entre_los_70_y_74_anios,proyeccion_de_poblacion_entre_los_75_y_79_anios
0,2016,5,0.927392,0.692244,0.535831,0.797418,0.655170,922.105440,1.394657e+07,103.519120,...,3583.664000,3144.192000,3234.088000,3190.184000,2708.352000,4200.104000,2108.064000,1561.576000,1083.008000,759.632000
1,2016,8,0.975304,0.698526,0.436748,0.560483,0.699874,1520.341739,8.604841e+06,63.870435,...,7670.000000,6547.217391,6425.347826,6260.304348,5133.478261,9337.521739,3904.869565,2870.826087,1938.347826,1434.695652
2,2016,11,0.956000,0.761400,0.763900,0.979100,0.748700,199661.450000,2.502023e+07,185.710000,...,628185.000000,547958.000000,518352.000000,497879.000000,423052.000000,600078.000000,325516.000000,241887.000000,167928.000000,107806.000000
3,2016,13,0.790543,0.647800,0.388726,0.576387,0.662178,596.764130,8.622052e+06,63.996957,...,2866.260870,2575.217391,2544.782609,2380.847826,1989.804348,4415.021739,1533.239130,1175.804348,821.847826,618.673913
4,2016,15,0.886130,0.676331,0.495280,0.782495,0.621669,180.033496,1.488473e+07,110.482114,...,639.471545,645.894309,650.121951,589.463415,510.154472,919.894309,421.682927,340.504065,258.016260,210.845528


In [145]:
sum_var_mun = merged_mun.groupby(to_group).sum().reset_index()
sum_var_mun.head()

,anio_corte,codigo_departamento,vacunacion_pentavalente_0_1,componente_de_resultados,componente_de_gestion,gobierno_abierto_y_transparencia,ejecucion_de_recursos,valor_agregado,valor_agregado_per_capita,valor_agregado_per_capita_como_porcentaje_del_promedio_nacional,...,proyeccion_de_poblacion_entre_los_35_y_39_anios,proyeccion_de_poblacion_entre_los_40_y_44_anios,proyeccion_de_poblacion_entre_los_45_y_49_anios,proyeccion_de_poblacion_entre_los_50_y_54_anios,proyeccion_de_poblacion_entre_los_55_y_59_anios,proyeccion_de_poblacion_entre_los_5_y_9_anios,proyeccion_de_poblacion_entre_los_60_y_64_anios,proyeccion_de_poblacion_entre_los_65_y_69_anios,proyeccion_de_poblacion_entre_los_70_y_74_anios,proyeccion_de_poblacion_entre_los_75_y_79_anios
0,2016,5,115.924,86.5305,66.9789,99.6773,81.8963,115263.18,1.743321e+09,12939.89,...,447958.0,393024.0,404261.0,398773.0,338544.0,525013.0,263508.0,195197.0,135376.0,94954.0
1,2016,8,22.432,16.0661,10.0452,12.8911,16.0971,34967.86,1.979113e+08,1469.02,...,176410.0,150586.0,147783.0,143987.0,118070.0,214763.0,89812.0,66029.0,44582.0,32998.0
2,2016,11,0.956,0.7614,0.7639,0.9791,0.7487,199661.45,2.502023e+07,185.71,...,628185.0,547958.0,518352.0,497879.0,423052.0,600078.0,325516.0,241887.0,167928.0,107806.0
3,2016,13,36.365,29.7988,17.8814,26.5138,30.4602,27451.15,3.966144e+08,2943.86,...,131848.0,118460.0,117060.0,109519.0,91531.0,203091.0,70529.0,54087.0,37805.0,28459.0
4,2016,15,108.994,83.1887,60.9194,96.2469,76.4653,22144.12,1.830821e+09,13589.30,...,78655.0,79445.0,79965.0,72504.0,62749.0,113147.0,51867.0,41882.0,31736.0,25934.0


In [152]:
var_mun = mean_var_mun.merge(sum_var_mun, on = to_group, suffixes = ('_mean', '_sum'))
var_mun.rename(columns = {'codigo_departamento': 'departame_cod'}, inplace = True)
var_mun.departame_cod = var_mun.departame_cod.astype('str')
var_mun.head()

,anio_corte,departame_cod,vacunacion_pentavalente_0_1_mean,componente_de_resultados_mean,componente_de_gestion_mean,gobierno_abierto_y_transparencia_mean,ejecucion_de_recursos_mean,valor_agregado_mean,valor_agregado_per_capita_mean,valor_agregado_per_capita_como_porcentaje_del_promedio_nacional_mean,...,proyeccion_de_poblacion_entre_los_35_y_39_anios_sum,proyeccion_de_poblacion_entre_los_40_y_44_anios_sum,proyeccion_de_poblacion_entre_los_45_y_49_anios_sum,proyeccion_de_poblacion_entre_los_50_y_54_anios_sum,proyeccion_de_poblacion_entre_los_55_y_59_anios_sum,proyeccion_de_poblacion_entre_los_5_y_9_anios_sum,proyeccion_de_poblacion_entre_los_60_y_64_anios_sum,proyeccion_de_poblacion_entre_los_65_y_69_anios_sum,proyeccion_de_poblacion_entre_los_70_y_74_anios_sum,proyeccion_de_poblacion_entre_los_75_y_79_anios_sum
0,2016,5,0.927392,0.692244,0.535831,0.797418,0.655170,922.105440,1.394657e+07,103.519120,...,447958.0,393024.0,404261.0,398773.0,338544.0,525013.0,263508.0,195197.0,135376.0,94954.0
1,2016,8,0.975304,0.698526,0.436748,0.560483,0.699874,1520.341739,8.604841e+06,63.870435,...,176410.0,150586.0,147783.0,143987.0,118070.0,214763.0,89812.0,66029.0,44582.0,32998.0
2,2016,11,0.956000,0.761400,0.763900,0.979100,0.748700,199661.450000,2.502023e+07,185.710000,...,628185.0,547958.0,518352.0,497879.0,423052.0,600078.0,325516.0,241887.0,167928.0,107806.0
3,2016,13,0.790543,0.647800,0.388726,0.576387,0.662178,596.764130,8.622052e+06,63.996957,...,131848.0,118460.0,117060.0,109519.0,91531.0,203091.0,70529.0,54087.0,37805.0,28459.0
4,2016,15,0.886130,0.676331,0.495280,0.782495,0.621669,180.033496,1.488473e+07,110.482114,...,78655.0,79445.0,79965.0,72504.0,62749.0,113147.0,51867.0,41882.0,31736.0,25934.0


In [166]:
to_merge = ['anio_corte', 'departame_cod']
var_dep = merge_dep.merge(var_mun, on = to_merge)
var_dep.head()

,anio_corte,departame_cod,departame_nombre,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,mdm,conectividad,transformacion,inversion_total,anio_inversion,vacunacion_pentavalente_0_1_mean,componente_de_resultados_mean,componente_de_gestion_mean,gobierno_abierto_y_transparencia_mean,ejecucion_de_recursos_mean,valor_agregado_mean,valor_agregado_per_capita_mean,valor_agregado_per_capita_como_porcentaje_del_promedio_nacional_mean,tasa_de_violencias_basadas_en_genero_mean,%_de_ingresos_corrientes_destinados_a_funcionamiento_mean,%_del_gasto_total_destinado_a_inversion_mean,capacidad_de_ahorro_mean,deficit_o_superavit_total_mean,financiamiento_mean,funcionamiento_mean,gastos_corrientes_mean,gastos_corrientes_per_capita_mean,gastos_de_capital_(inversion)_mean,gastos_de_funcionamiento_per_capita_mean,gastos_totales_mean,gastos_totales_per_capita_mean,indicador_de_desempenio_fiscal_mean,ingresos_corrientes_mean,ingresos_de_capital_mean,ingresos_no_tributarios_mean,ingresos_no_tributarios_per_capita_mean,ingresos_per_capita_por_impuesto_a_la_industria_y_al_comercio_mean,ingresos_per_capita_por_impuesto_predial_mean,ingresos_totales_mean,ingresos_totales_per_capita_mean,ingresos_tributarios_mean,ingresos_tributarios_per_capita_mean,intereses_de_deuda_publica_mean,recursos_del_balance_y_variacion_de_depositos_mean,regalias_per_capita_(valor_efectivamente_girado_al_municipio)_mean,total_de_recaudo_de_ingresos_corrientes_per_capita_mean,transferencias_de_los_ingresos_corrientes_mean,transferencias_nacionales_(sgp;_etc.)_per_capita__mean,transferencias_per_capita_de_los_ingresos_corrientes_mean,cobertura_bruta_en_educacion_-_total_mean,cobertura_bruta_en_educacion_basica_mean,cobertura_bruta_en_educacion_media_mean,cobertura_bruta_en_educacion_primaria_mean,cobertura_bruta_en_educacion_secundaria_mean,cobertura_bruta_en_transicion_mean,cobertura_neta_en_educacion_-_total_mean,cobertura_neta_en_educacion_basica_mean,cobertura_neta_en_educacion_media_mean,cobertura_neta_en_educacion_primaria_mean,cobertura_neta_en_educacion_secundaria_mean,cobertura_neta_en_transicion_mean,puntaje_promedio_pruebas_saber_11_-_lectura_critica_mean,puntaje_promedio_pruebas_saber_11_-_matematicas_mean,tasa_de_desercion_intra-anual_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,tasa_de_repitencia_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,afiliados_a_regimenes_especiales_mean,afiliados_al_regimen_contributivo_mean,afiliados_al_regimen_subsidado_mean,afiliados_al_sgsss_mean,cobertura_del_regimen_subsidiado_mean,mortalidad_en_menores_de_5_anios_por_enfermedad_diarreica_aguda_(eda)_mean,mortalidad_en_menores_de_5_anios_por_infeccion_respiratoria_aguda_(ira)_mean,porcentaje_de_nacidos_vivos_con_4_o_mas_consultas_de_control_prenatal_mean,porcentaje_de_nacidos_vivos_con_bajo_peso_al_nacer_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_14_a_17_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_18_a_26_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_menores_de_14_anios_mean,promedio_de_controles_prenatales_mean,razon_de_mortalidad_materna_a_42_dias_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_14_anios_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_19_anios_mean,tasa_de_mortalidad_(x_cada_1.000_habitantes)_mean,tasa_de_mortalidad_infantil_en_menores_de_1_anio_(x_cada_1.000_nacidos_vivos)_mean,tasa_de_mortalidad_infantil_en_menores_de_5_anios__mean,cobertura_de_acueducto_rural_(rec)_mean,cobertura_de_acueducto_urbana_(rec)_mean,cobertura_de_alcantarillado_rural_(rec)_mean,cobertura_de_alcantarillado_urbana_(rec)_mean,cobertura_de_aseo_(rec)_mean,cobertura_de_aseo_rural

In [167]:
var_dep.to_csv('../data/var_dep.csv', index=False)

In [168]:
to_drop = ['anio_inversion', 'valor_agregado_per_capita_mean', 'valor_agregado_per_capita_como_porcentaje_del_promedio_nacional_mean',
           'gastos_corrientes_per_capita_mean', 'gastos_de_funcionamiento_per_capita_mean', 'gastos_totales_per_capita_mean',
           'ingresos_no_tributarios_per_capita_mean', 'ingresos_per_capita_por_impuesto_a_la_industria_y_al_comercio_mean',
           'ingresos_per_capita_por_impuesto_predial_mean', 'ingresos_totales_per_capita_mean', 'ingresos_tributarios_per_capita_mean',
           'total_de_recaudo_de_ingresos_corrientes_per_capita_mean', 'transferencias_nacionales_(sgp;_etc.)_per_capita__mean',
           'transferencias_per_capita_de_los_ingresos_corrientes_mean', 'cobertura_bruta_en_transicion_mean', 
           'cobertura_bruta_en_educacion_secundaria_mean', 'cobertura_bruta_en_educacion_primaria_mean', 'cobertura_bruta_en_educacion_media_mean', 
           'cobertura_bruta_en_educacion_basica_mean', 'cobertura_bruta_en_educacion_-_total_mean', 
           'puntaje_promedio_pruebas_saber_11_-_lectura_critica_mean', 'puntaje_promedio_pruebas_saber_11_-_matematicas_mean', 
           'tasa_de_mortalidad_(x_cada_1.000_habitantes)_mean', 'tasa_de_mortalidad_infantil_en_menores_de_1_anio_(x_cada_1.000_nacidos_vivos)_mean',
           'tasa_de_mortalidad_infantil_en_menores_de_5_anios__mean', 'cobertura_de_acueducto_rural_(rec)_mean', 
           'cobertura_de_acueducto_urbana_(rec)_mean', 'cobertura_de_alcantarillado_rural_(rec)_mean', 'cobertura_de_alcantarillado_urbana_(rec)_mean', 
           'cobertura_de_aseo_(rec)_mean', 'cobertura_de_aseo_rural_(rec)_mean', 'cobertura_de_aseo_urbana_(rec)_mean', 'densidad_poblacional_mean', 
           'hombres_mayores_de_edad_mean', 'hombres_menores_de_edad_mean', 'mujeres_mayores_de_edad_mean', 'mujeres_menores_de_edad_mean', 
           'poblacion_de_hombres_de_00-04_mean', 'poblacion_de_hombres_de_80_o_mas_mean', 'poblacion_de_hombres_de__05-09_mean', 
           'poblacion_de_hombres_de__10-14_mean', 'poblacion_de_hombres_de__15-19_mean', 'poblacion_de_hombres_de__20-24_mean', 
           'poblacion_de_hombres_de__25-29_mean', 'poblacion_de_hombres_de__30-34_mean', 'poblacion_de_hombres_de__35-39_mean', 
           'poblacion_de_hombres_de__40-44_mean', 'poblacion_de_hombres_de__45-49_mean', 'poblacion_de_hombres_de__50-54_mean', 
           'poblacion_de_hombres_de__55-59_mean', 'poblacion_de_hombres_de__60-64_mean', 'poblacion_de_hombres_de__65-69_mean', 
           'poblacion_de_hombres_de__70-74_mean', 'poblacion_de_hombres_de__75-79_mean', 'poblacion_de_mujeres_de_00-04_mean', 
           'poblacion_de_mujeres_de_80_o_mas_mean', 'poblacion_de_mujeres_de__05-09_mean', 'poblacion_de_mujeres_de__10-14_mean', 
           'poblacion_de_mujeres_de__15-19_mean', 'poblacion_de_mujeres_de__20-24_mean', 'poblacion_de_mujeres_de__25-29_mean', 
           'poblacion_de_mujeres_de__30-34_mean', 'poblacion_de_mujeres_de__35-39_mean', 'poblacion_de_mujeres_de__40-44_mean', 
           'poblacion_de_mujeres_de__45-49_mean', 'poblacion_de_mujeres_de__50-54_mean', 'poblacion_de_mujeres_de__55-59_mean', 
           'poblacion_de_mujeres_de__60-64_mean', 'poblacion_de_mujeres_de__65-69_mean', 'poblacion_de_mujeres_de__70-74_mean', 
           'poblacion_de_mujeres_de__75-79_mean', 'poblacion_rural_mean', 'poblacion_total_mean', 'poblacion_total_de_hombres_mean', 
           'poblacion_total_de_mujeres_mean', 'poblacion_total_mayor_de_edad_mean', 'poblacion_total_menor_de_edad_mean', 'poblacion_urbana_mean', 
           'porcentaje_de_hombres_mayores_de_edad_mean', 'porcentaje_de_hombres_menores_de_edad_mean', 
           'porcentaje_de_la_poblacion_total__menor_de_edad_mean', 'porcentaje_de_la_poblacion_total_mayor_de_edad_mean', 
           'porcentaje_de_mujeres_mayores_de_edad_mean', 'porcentaje_de_mujeres_menores_de_edad_mean', 
           'porcentaje_de_poblacion_de_80_o_mas_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_de_hombres_de_00-04_mean', 'porcentaje_de_poblacion_de_hombres_de_80_o_mas_mean', 
           'porcentaje_de_poblacion_de_hombres_de__05-09_mean', 'porcentaje_de_poblacion_de_hombres_de__10-14_mean', 
           'porcentaje_de_poblacion_de_hombres_de__15-19_mean', 'porcentaje_de_poblacion_de_hombres_de__20-24_mean', 
           'porcentaje_de_poblacion_de_hombres_de__25-29_mean', 'porcentaje_de_poblacion_de_hombres_de__30-34_mean', 
           'porcentaje_de_poblacion_de_hombres_de__35-39_mean', 'porcentaje_de_poblacion_de_hombres_de__40-44_mean', 
           'porcentaje_de_poblacion_de_hombres_de__45-49_mean', 'porcentaje_de_poblacion_de_hombres_de__50-54_mean', 
           'porcentaje_de_poblacion_de_hombres_de__55-59_mean', 'porcentaje_de_poblacion_de_hombres_de__60-64_mean', 
           'porcentaje_de_poblacion_de_hombres_de__65-69_mean', 'porcentaje_de_poblacion_de_hombres_de__70-74_mean', 
           'porcentaje_de_poblacion_de_hombres_de__75-79_mean', 'porcentaje_de_poblacion_de_mujeres_de_00-04_mean', 
           'porcentaje_de_poblacion_de_mujeres_de_80_o_mas_mean', 'porcentaje_de_poblacion_de_mujeres_de__05-09_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__10-14_mean', 'porcentaje_de_poblacion_de_mujeres_de__15-19_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__20-24_mean', 'porcentaje_de_poblacion_de_mujeres_de__25-29_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__30-34_mean', 'porcentaje_de_poblacion_de_mujeres_de__35-39_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__40-44_mean', 'porcentaje_de_poblacion_de_mujeres_de__45-49_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__50-54_mean', 'porcentaje_de_poblacion_de_mujeres_de__55-59_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__60-64_mean', 'porcentaje_de_poblacion_de_mujeres_de__65-69_mean', 
           'porcentaje_de_poblacion_de_mujeres_de__70-74_mean', 'porcentaje_de_poblacion_de_mujeres_de__75-79_mean', 
           'porcentaje_de_poblacion_entre_los_0_y_4_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_10_y_14_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_15_y_19_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_20_y_24_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_25_y_29_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_30_y_34_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_35_y_39_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_40_y_44_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_45_y_49_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_50_y_54_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_55_y_59_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_5_y_9_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_60_y_64_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_65_y_69_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_70_y_74_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_de_poblacion_entre_los_75_y_79_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_mean', 
           'porcentaje_poblacion_de_hombres_mean', 'porcentaje_poblacion_de_mujeres_mean', 'porcentaje_poblacion_rural_mean', 
           'porcentaje_poblacion_urbana_mean', 'proyeccion_de_poblacion_de_80_o_mas_anios_mean', 'proyeccion_de_poblacion_entre_los_0_y_4_anios_mean', 
           'proyeccion_de_poblacion_entre_los_10_y_14_anios_mean', 'proyeccion_de_poblacion_entre_los_15_y_19_anios_mean', 
           'proyeccion_de_poblacion_entre_los_20_y_24_anios_mean', 'proyeccion_de_poblacion_entre_los_25_y_29_anios_mean', 
           'proyeccion_de_poblacion_entre_los_30_y_34_anios_mean', 'proyeccion_de_poblacion_entre_los_35_y_39_anios_mean', 
           'proyeccion_de_poblacion_entre_los_40_y_44_anios_mean', 'proyeccion_de_poblacion_entre_los_45_y_49_anios_mean', 
           'proyeccion_de_poblacion_entre_los_50_y_54_anios_mean', 'proyeccion_de_poblacion_entre_los_55_y_59_anios_mean', 
           'proyeccion_de_poblacion_entre_los_5_y_9_anios_mean', 'proyeccion_de_poblacion_entre_los_60_y_64_anios_mean', 
           'proyeccion_de_poblacion_entre_los_65_y_69_anios_mean', 'proyeccion_de_poblacion_entre_los_70_y_74_anios_mean', 
           'proyeccion_de_poblacion_entre_los_75_y_79_anios_mean']
var_dep.drop(to_drop, axis = 'columns', inplace = True)
var_dep.head()

,anio_corte,departame_cod,departame_nombre,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,mdm,conectividad,transformacion,inversion_total,vacunacion_pentavalente_0_1_mean,componente_de_resultados_mean,componente_de_gestion_mean,gobierno_abierto_y_transparencia_mean,ejecucion_de_recursos_mean,valor_agregado_mean,tasa_de_violencias_basadas_en_genero_mean,%_de_ingresos_corrientes_destinados_a_funcionamiento_mean,%_del_gasto_total_destinado_a_inversion_mean,capacidad_de_ahorro_mean,deficit_o_superavit_total_mean,financiamiento_mean,funcionamiento_mean,gastos_corrientes_mean,gastos_de_capital_(inversion)_mean,gastos_totales_mean,indicador_de_desempenio_fiscal_mean,ingresos_corrientes_mean,ingresos_de_capital_mean,ingresos_no_tributarios_mean,ingresos_totales_mean,ingresos_tributarios_mean,intereses_de_deuda_publica_mean,recursos_del_balance_y_variacion_de_depositos_mean,regalias_per_capita_(valor_efectivamente_girado_al_municipio)_mean,transferencias_de_los_ingresos_corrientes_mean,cobertura_neta_en_educacion_-_total_mean,cobertura_neta_en_educacion_basica_mean,cobertura_neta_en_educacion_media_mean,cobertura_neta_en_educacion_primaria_mean,cobertura_neta_en_educacion_secundaria_mean,cobertura_neta_en_transicion_mean,tasa_de_desercion_intra-anual_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,tasa_de_repitencia_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,afiliados_a_regimenes_especiales_mean,afiliados_al_regimen_contributivo_mean,afiliados_al_regimen_subsidado_mean,afiliados_al_sgsss_mean,cobertura_del_regimen_subsidiado_mean,mortalidad_en_menores_de_5_anios_por_enfermedad_diarreica_aguda_(eda)_mean,mortalidad_en_menores_de_5_anios_por_infeccion_respiratoria_aguda_(ira)_mean,porcentaje_de_nacidos_vivos_con_4_o_mas_consultas_de_control_prenatal_mean,porcentaje_de_nacidos_vivos_con_bajo_peso_al_nacer_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_14_a_17_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_18_a_26_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_menores_de_14_anios_mean,promedio_de_controles_prenatales_mean,razon_de_mortalidad_materna_a_42_dias_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_14_anios_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_19_anios_mean,vacunacion_pentavalente_0_1_sum,componente_de_resultados_sum,componente_de_gestion_sum,gobierno_abierto_y_transparencia_sum,ejecucion_de_recursos_sum,valor_agregado_sum,valor_agregado_per_capita_sum,valor_agregado_per_capita_como_porcentaje_del_promedio_nacional_sum,tasa_de_violencias_basadas_en_genero_sum,%_de_ingresos_corrientes_destinados_a_funcionamiento_sum,%_del_gasto_total_destinado_a_inversion_sum,capacidad_de_ahorro_sum,deficit_o_superavit_total_sum,financiamiento_sum,funcionamiento_sum,gastos_corrientes_sum,gastos_corrientes_per_capita_sum,gastos_de_capital_(inversion)_sum,gastos_de_funcionamiento_per_capita_sum,gastos_totales_sum,gastos_totales_per_capita_sum,indicador_de_desempenio_fiscal_sum,ingresos_corrientes_sum,ingresos_de_capital_sum,ingresos_no_tributarios_sum,ingresos_no_tributarios_per_capita_sum,ingresos_per_capita_por_impuesto_a_la_industria_y_al_comercio_sum,ingresos_per_capita_por_impuesto_predial_sum,ingresos_totales_sum,ingresos_totales_per_capita_sum,ingresos_tributarios_sum,ingresos_tributarios_per_capita_sum,intereses_de_deuda_publica_sum,recursos_del_balance_y_variacion_de_depositos_sum,regalias_per_capita_(valor_efectivamente_girado_al_municipio)_sum,total_de_recaudo_de_ingresos_corrientes_per_capita_sum,transferencias_de_los_ingresos_corrientes_sum,transferencias_nacionales_(sgp;_etc.)_per_capita__sum,transferencias_per_capita_

In [169]:
to_drop = ['vacunacion_pentavalente_0_1_sum', 'componente_de_resultados_sum', 'componente_de_gestion_sum', 'gobierno_abierto_y_transparencia_sum', 
           'ejecucion_de_recursos_sum', 'valor_agregado_per_capita_sum', 'valor_agregado_per_capita_como_porcentaje_del_promedio_nacional_sum', 
           'tasa_de_violencias_basadas_en_genero_sum', '%_de_ingresos_corrientes_destinados_a_funcionamiento_sum', 
           '%_del_gasto_total_destinado_a_inversion_sum', 'capacidad_de_ahorro_sum', 'gastos_corrientes_per_capita_sum', 
           'gastos_de_funcionamiento_per_capita_sum', 'gastos_totales_per_capita_sum', 'indicador_de_desempenio_fiscal_sum',
           'ingresos_no_tributarios_per_capita_sum', 'ingresos_per_capita_por_impuesto_a_la_industria_y_al_comercio_sum', 
           'ingresos_per_capita_por_impuesto_predial_sum', 'ingresos_totales_per_capita_sum', 'ingresos_tributarios_per_capita_sum', 
           'total_de_recaudo_de_ingresos_corrientes_per_capita_sum', 'transferencias_nacionales_(sgp;_etc.)_per_capita__sum', 
           'transferencias_per_capita_de_los_ingresos_corrientes_sum', 'cobertura_bruta_en_educacion_-_total_sum', 
           'cobertura_neta_en_educacion_-_total_sum', 'cobertura_neta_en_educacion_basica_sum', 'cobertura_neta_en_educacion_media_sum', 
           'cobertura_neta_en_educacion_primaria_sum', 'cobertura_neta_en_educacion_secundaria_sum', 'cobertura_neta_en_transicion_sum', 
           'puntaje_promedio_pruebas_saber_11_-_lectura_critica_sum', 'puntaje_promedio_pruebas_saber_11_-_matematicas_sum', 
           'tasa_de_desercion_intra-anual_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_sum', 
           'tasa_de_repitencia_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_sum', 
           'cobertura_del_regimen_subsidiado_sum', 'porcentaje_de_nacidos_vivos_con_4_o_mas_consultas_de_control_prenatal_sum', 
           'porcentaje_de_nacidos_vivos_con_bajo_peso_al_nacer_sum', 'porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_14_a_17_anios_sum', 
           'porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_18_a_26_anios_sum', 'porcentaje_de_nacidos_vivos_hijos_de_mujeres_menores_de_14_anios_sum', 
           'promedio_de_controles_prenatales_sum', 'razon_de_mortalidad_materna_a_42_dias_sum', 
           'tasa_de_fecundidad_especifica_en_mujeres_de_10_a_14_anios_sum', 'tasa_de_fecundidad_especifica_en_mujeres_de_10_a_19_anios_sum', 
           'tasa_de_mortalidad_(x_cada_1.000_habitantes)_sum', 'tasa_de_mortalidad_infantil_en_menores_de_1_anio_(x_cada_1.000_nacidos_vivos)_sum', 
           'tasa_de_mortalidad_infantil_en_menores_de_5_anios__sum', 'cobertura_de_acueducto_rural_(rec)_sum', 'cobertura_de_acueducto_urbana_(rec)_sum', 
           'cobertura_de_alcantarillado_rural_(rec)_sum', 'cobertura_de_alcantarillado_urbana_(rec)_sum', 'cobertura_de_aseo_(rec)_sum', 
           'cobertura_de_aseo_rural_(rec)_sum', 'cobertura_de_aseo_urbana_(rec)_sum', 'densidad_poblacional_sum', 'hombres_mayores_de_edad_sum', 
           'hombres_menores_de_edad_sum', 'mujeres_mayores_de_edad_sum', 'mujeres_menores_de_edad_sum', 'poblacion_de_hombres_de_00-04_sum', 
           'poblacion_de_hombres_de_80_o_mas_sum', 'poblacion_de_hombres_de__05-09_sum', 'poblacion_de_hombres_de__10-14_sum', 
           'poblacion_de_hombres_de__15-19_sum', 'poblacion_de_hombres_de__20-24_sum', 'poblacion_de_hombres_de__25-29_sum', 
           'poblacion_de_hombres_de__30-34_sum', 'poblacion_de_hombres_de__35-39_sum', 'poblacion_de_hombres_de__40-44_sum', 
           'poblacion_de_hombres_de__45-49_sum', 'poblacion_de_hombres_de__50-54_sum', 'poblacion_de_hombres_de__55-59_sum', 
           'poblacion_de_hombres_de__60-64_sum', 'poblacion_de_hombres_de__65-69_sum', 'poblacion_de_hombres_de__70-74_sum', 
           'poblacion_de_hombres_de__75-79_sum', 'poblacion_de_mujeres_de_00-04_sum', 'poblacion_de_mujeres_de_80_o_mas_sum', 
           'poblacion_de_mujeres_de__05-09_sum', 'poblacion_de_mujeres_de__10-14_sum', 'poblacion_de_mujeres_de__15-19_sum', 
           'poblacion_de_mujeres_de__20-24_sum', 'poblacion_de_mujeres_de__25-29_sum', 'poblacion_de_mujeres_de__30-34_sum', 
           'poblacion_de_mujeres_de__35-39_sum', 'poblacion_de_mujeres_de__40-44_sum', 'poblacion_de_mujeres_de__45-49_sum', 
           'poblacion_de_mujeres_de__50-54_sum', 'poblacion_de_mujeres_de__55-59_sum', 'poblacion_de_mujeres_de__60-64_sum', 
           'poblacion_de_mujeres_de__65-69_sum', 'poblacion_de_mujeres_de__70-74_sum', 'poblacion_de_mujeres_de__75-79_sum', 
           'poblacion_rural_sum', 'poblacion_total_sum', 'poblacion_total_de_hombres_sum', 'poblacion_total_de_mujeres_sum', 
           'poblacion_total_mayor_de_edad_sum', 'poblacion_total_menor_de_edad_sum', 'poblacion_urbana_sum', 'porcentaje_de_hombres_mayores_de_edad_sum', 
           'porcentaje_de_hombres_menores_de_edad_sum', 'porcentaje_de_la_poblacion_total__menor_de_edad_sum', 
           'porcentaje_de_la_poblacion_total_mayor_de_edad_sum', 'porcentaje_de_mujeres_mayores_de_edad_sum', 'porcentaje_de_mujeres_menores_de_edad_sum', 
           'porcentaje_de_poblacion_de_80_o_mas_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_de_hombres_de_00-04_sum', 'porcentaje_de_poblacion_de_hombres_de_80_o_mas_sum', 
           'porcentaje_de_poblacion_de_hombres_de__05-09_sum', 'porcentaje_de_poblacion_de_hombres_de__10-14_sum', 
           'porcentaje_de_poblacion_de_hombres_de__15-19_sum', 'porcentaje_de_poblacion_de_hombres_de__20-24_sum', 
           'porcentaje_de_poblacion_de_hombres_de__25-29_sum', 'porcentaje_de_poblacion_de_hombres_de__30-34_sum', 
           'porcentaje_de_poblacion_de_hombres_de__35-39_sum', 'porcentaje_de_poblacion_de_hombres_de__40-44_sum', 
           'porcentaje_de_poblacion_de_hombres_de__45-49_sum', 'porcentaje_de_poblacion_de_hombres_de__50-54_sum', 
           'porcentaje_de_poblacion_de_hombres_de__55-59_sum', 'porcentaje_de_poblacion_de_hombres_de__60-64_sum', 
           'porcentaje_de_poblacion_de_hombres_de__65-69_sum', 'porcentaje_de_poblacion_de_hombres_de__70-74_sum', 
           'porcentaje_de_poblacion_de_hombres_de__75-79_sum', 'porcentaje_de_poblacion_de_mujeres_de_00-04_sum', 
           'porcentaje_de_poblacion_de_mujeres_de_80_o_mas_sum', 'porcentaje_de_poblacion_de_mujeres_de__05-09_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__10-14_sum', 'porcentaje_de_poblacion_de_mujeres_de__15-19_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__20-24_sum', 'porcentaje_de_poblacion_de_mujeres_de__25-29_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__30-34_sum', 'porcentaje_de_poblacion_de_mujeres_de__35-39_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__40-44_sum', 'porcentaje_de_poblacion_de_mujeres_de__45-49_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__50-54_sum', 'porcentaje_de_poblacion_de_mujeres_de__55-59_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__60-64_sum', 'porcentaje_de_poblacion_de_mujeres_de__65-69_sum', 
           'porcentaje_de_poblacion_de_mujeres_de__70-74_sum', 'porcentaje_de_poblacion_de_mujeres_de__75-79_sum', 
           'porcentaje_de_poblacion_entre_los_0_y_4_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_10_y_14_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_15_y_19_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_20_y_24_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_25_y_29_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_30_y_34_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_35_y_39_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_40_y_44_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_45_y_49_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_50_y_54_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_55_y_59_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_5_y_9_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_60_y_64_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_65_y_69_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_70_y_74_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_de_poblacion_entre_los_75_y_79_anios;_del_total_de_poblacion_proyectada_para_la_entidad_territorial_sum', 
           'porcentaje_poblacion_de_hombres_sum', 'porcentaje_poblacion_de_mujeres_sum', 'porcentaje_poblacion_rural_sum', 
           'porcentaje_poblacion_urbana_sum', 'proyeccion_de_poblacion_de_80_o_mas_anios_sum', 'proyeccion_de_poblacion_entre_los_0_y_4_anios_sum', 
           'proyeccion_de_poblacion_entre_los_10_y_14_anios_sum', 'proyeccion_de_poblacion_entre_los_15_y_19_anios_sum', 
           'proyeccion_de_poblacion_entre_los_20_y_24_anios_sum', 'proyeccion_de_poblacion_entre_los_25_y_29_anios_sum', 
           'proyeccion_de_poblacion_entre_los_30_y_34_anios_sum', 'proyeccion_de_poblacion_entre_los_35_y_39_anios_sum', 
           'proyeccion_de_poblacion_entre_los_40_y_44_anios_sum', 'proyeccion_de_poblacion_entre_los_45_y_49_anios_sum', 
           'proyeccion_de_poblacion_entre_los_50_y_54_anios_sum', 'proyeccion_de_poblacion_entre_los_55_y_59_anios_sum', 
           'proyeccion_de_poblacion_entre_los_5_y_9_anios_sum', 'proyeccion_de_poblacion_entre_los_60_y_64_anios_sum', 
           'proyeccion_de_poblacion_entre_los_65_y_69_anios_sum', 'proyeccion_de_poblacion_entre_los_70_y_74_anios_sum', 
           'proyeccion_de_poblacion_entre_los_75_y_79_anios_sum']
var_dep.drop(to_drop, axis = 'columns', inplace = True)
var_dep.head()

,anio_corte,departame_cod,departame_nombre,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,mdm,conectividad,transformacion,inversion_total,vacunacion_pentavalente_0_1_mean,componente_de_resultados_mean,componente_de_gestion_mean,gobierno_abierto_y_transparencia_mean,ejecucion_de_recursos_mean,valor_agregado_mean,tasa_de_violencias_basadas_en_genero_mean,%_de_ingresos_corrientes_destinados_a_funcionamiento_mean,%_del_gasto_total_destinado_a_inversion_mean,capacidad_de_ahorro_mean,deficit_o_superavit_total_mean,financiamiento_mean,funcionamiento_mean,gastos_corrientes_mean,gastos_de_capital_(inversion)_mean,gastos_totales_mean,indicador_de_desempenio_fiscal_mean,ingresos_corrientes_mean,ingresos_de_capital_mean,ingresos_no_tributarios_mean,ingresos_totales_mean,ingresos_tributarios_mean,intereses_de_deuda_publica_mean,recursos_del_balance_y_variacion_de_depositos_mean,regalias_per_capita_(valor_efectivamente_girado_al_municipio)_mean,transferencias_de_los_ingresos_corrientes_mean,cobertura_neta_en_educacion_-_total_mean,cobertura_neta_en_educacion_basica_mean,cobertura_neta_en_educacion_media_mean,cobertura_neta_en_educacion_primaria_mean,cobertura_neta_en_educacion_secundaria_mean,cobertura_neta_en_transicion_mean,tasa_de_desercion_intra-anual_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,tasa_de_repitencia_del_sector_oficial_en_educacion_basica_y_media_(desde_transicion_hasta_once)_mean,afiliados_a_regimenes_especiales_mean,afiliados_al_regimen_contributivo_mean,afiliados_al_regimen_subsidado_mean,afiliados_al_sgsss_mean,cobertura_del_regimen_subsidiado_mean,mortalidad_en_menores_de_5_anios_por_enfermedad_diarreica_aguda_(eda)_mean,mortalidad_en_menores_de_5_anios_por_infeccion_respiratoria_aguda_(ira)_mean,porcentaje_de_nacidos_vivos_con_4_o_mas_consultas_de_control_prenatal_mean,porcentaje_de_nacidos_vivos_con_bajo_peso_al_nacer_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_14_a_17_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_de_18_a_26_anios_mean,porcentaje_de_nacidos_vivos_hijos_de_mujeres_menores_de_14_anios_mean,promedio_de_controles_prenatales_mean,razon_de_mortalidad_materna_a_42_dias_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_14_anios_mean,tasa_de_fecundidad_especifica_en_mujeres_de_10_a_19_anios_mean,valor_agregado_sum,deficit_o_superavit_total_sum,financiamiento_sum,funcionamiento_sum,gastos_corrientes_sum,gastos_de_capital_(inversion)_sum,gastos_totales_sum,ingresos_corrientes_sum,ingresos_de_capital_sum,ingresos_no_tributarios_sum,ingresos_totales_sum,ingresos_tributarios_sum,intereses_de_deuda_publica_sum,recursos_del_balance_y_variacion_de_depositos_sum,regalias_per_capita_(valor_efectivamente_girado_al_municipio)_sum,transferencias_de_los_ingresos_corrientes_sum,cobertura_bruta_en_educacion_basica_sum,cobertura_bruta_en_educacion_media_sum,cobertura_bruta_en_educacion_primaria_sum,cobertura_bruta_en_educacion_secundaria_sum,cobertura_bruta_en_transicion_sum,afiliados_a_regimenes_especiales_sum,afiliados_al_regimen_contributivo_sum,afiliados_al_regimen_subsidado_sum,afiliados_al_sgsss_sum,mortalidad_en_menores_de_5_anios_por_enfermedad_diarreica_aguda_(eda)_sum,mortalidad_en_menores_de_5_anios_por_infeccion_respiratoria_aguda_(ira)_sum
0,2016,11,bogota,0.497000,54.090000,54.350000,0.564000,0.924000,0.956000,11.730000,1.000000,0.211000,0.960000,0.960000,34364.000000,43.060000,1278.000000,1.600000,4738.000000,5.940000,77.560000,3.916084e+09,2.094678e+10,2.486286e+10,0.956000,0.761400,0.763900,0.979100,0.748700,199661.450000,281.710000,33.350000,74.360000,60.550000,-84167.740000,84167.740000,3.214225e+06,3.518697e+06,1.020408e+07,1.372278e+07,82.500000,8

In [170]:
len(var_dep)

132

In [171]:
len(var_dep.columns)

101

In [172]:
var_dep.to_csv('../data/var_dep_filtered.csv', index=False)